<a href="https://colab.research.google.com/github/Waseem21Saleem/Cloud-Computing/blob/main/Homework/HW2/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase
!pip install firebase-admin ipywidgets plotly pandas
#!pip install --upgrade plotly
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.8 MB/s eta 0:00:00


In [4]:
import firebase_admin
from firebase_admin import credentials, db
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import io
import base64
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader


matplotlib.use('Agg')  # Use a non-interactive backend for matplotlib

# Initialize Firebase app with JSON contents
def init_firebase(json_contents):
    try:
        firebase_admin.get_app()
    except ValueError:
        cred = credentials.Certificate(json_contents)
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://phoenixes-34c33-default-rtdb.europe-west1.firebasedatabase.app/'
        })

# Fetch data from Firebase
def fetch_data():
    ref = db.reference('/')
    data = ref.get()
    if not data:
        raise ValueError("No data found in the Firebase database.")
    return data

# Process data
def process_data(data):
    relevant_actions = []
    for entry in data:
        description = entry.get('Description', '')
        if 'opened by' in description or 'Change Element Timings Event' in description:
            relevant_actions.append(entry)
    if not relevant_actions:
        raise ValueError("No relevant actions found in the data.")
    df = pd.DataFrame(relevant_actions)
    df['Time'] = pd.to_datetime(df['Time'])
    return df

# Functions for data processing and visualization
def filter_data_by_date(df, date):
    if date:
        date_str = date.strftime('%Y-%m-%d')
        filtered_df = df[df['Time'].dt.strftime('%Y-%m-%d') == date_str]
        return filtered_df
    return df

def filter_data_by_month(df, month):
    if month:
        month_str = month.strftime('%Y-%m')
        filtered_df = df[df['Time'].dt.strftime('%Y-%m') == month_str]
        return filtered_df
    return df

def calculate_progress_percentage(filtered_df):
    student_actions = filtered_df['User'].value_counts()
    total_actions = student_actions.sum()
    progress_percentage = (student_actions / total_actions) * 100
    return progress_percentage

def create_student_progress_charts(filtered_df):
    progress_percentage = calculate_progress_percentage(filtered_df)
    figures = []
    for student, percentage in progress_percentage.items():
        student_df = filtered_df[filtered_df['User'] == student]
        progress_counts = student_df['Description'].value_counts()
        fig, ax = plt.subplots(figsize=(10, 6))  # Adjust size for better readability
        ax.barh(progress_counts.index, progress_counts.values, color='skyblue')
        ax.set_title(f"Progress for {student} ({percentage:.2f}%)")
        ax.set_xlabel("Count")
        ax.set_ylabel("Actions")
        plt.tight_layout()
        figures.append(fig)
    return figures

def create_monthly_summary_chart(filtered_df):
    progress_percentage = calculate_progress_percentage(filtered_df)
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(progress_percentage.index, progress_percentage.values, color='skyblue')

    # Adding percentage labels above bars
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}%', va='bottom')  # va: vertical alignment

    ax.set_title("Monthly Summary")
    ax.set_xlabel("Students")
    ax.set_ylabel("Progress Percentage")
    plt.tight_layout()
    return fig

def find_top_performer(filtered_df):
    progress_percentage = calculate_progress_percentage(filtered_df)
    top_student = progress_percentage.idxmax()
    top_percentage = progress_percentage.max()
    return top_student, top_percentage

def generate_report(student_name, from_date, to_date):
    # Convert date strings to datetime objects
    from_date = pd.to_datetime(from_date)
    to_date = pd.to_datetime(to_date)

    # Filter data for the specified student and date range
    student_df = df[(df['User'] == student_name) & (df['Time'] >= from_date) & (df['Time'] <= to_date)]

    # Create a PDF buffer
    pdf_buffer = io.BytesIO()
    c = canvas.Canvas(pdf_buffer, pagesize=letter)

    # Add a centered title
    c.setFont("Helvetica-Bold", 18)
    c.drawCentredString(300, 750, "Performance Report")

    # Add student name and date range information in bold
    c.setFont("Helvetica-Bold", 12)
    c.drawString(100, 700, f"Student Name: {student_name}")
    c.drawString(100, 680, f"From Date: {from_date.strftime('%Y-%m-%d')}")
    c.drawString(100, 660, f"To Date: {to_date.strftime('%Y-%m-%d')}")
    c.setFont("Helvetica", 12)
    c.drawString(100, 640, "Generated using the Student Progress Dashboard.")

    # Add a summary of actions
    if not student_df.empty:
        c.setFont("Helvetica-Bold", 12)
        c.drawString(100, 620, "Summary of Actions:")
        c.setFont("Helvetica", 12)
        action_counts = student_df['Description'].value_counts()
        y_position = 600
        for action, count in action_counts.items():
            c.drawString(100, y_position, f"{action}: {count} times")
            y_position -= 20
            if y_position < 50:
                c.showPage()
                y_position = 750
    else:
        c.drawString(100, 620, "No actions found for the specified date range.")

    # Save and close the PDF
    c.showPage()
    c.save()

    # Get the PDF data
    pdf_buffer.seek(0)
    pdf_bytes = pdf_buffer.read()

    # Encode the PDF in base64
    b64_pdf = base64.b64encode(pdf_bytes).decode('utf-8')

    return b64_pdf


# Register the callback
#output.register_callback('generateReport', generate_report)



def fig_to_html(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    return f'<img src="data:image/png;base64,{img_str}" class="chart" />'

# Callback function to update the GUI
def update_gui(tab, date):
    global df  # Use the global dataframe
    output_html = ""

    if tab == 'Overall Progress':
        filtered_df = filter_data_by_date(df, pd.to_datetime(date))
        if filtered_df.empty:
            output_html = "<h3>No data available for the selected date.</h3>"
        else:
            progress_percentage = calculate_progress_percentage(filtered_df)
            output_html += "<h3>Overall Progress Percentage</h3><p>This chart shows the percentage of actions taken by each student out of the total actions recorded on the selected date.</p>"
            fig, ax = plt.subplots()
            ax.bar(progress_percentage.index, progress_percentage.values)
            ax.set_title("Overall Progress Percentage")
            ax.set_xlabel("Students")
            ax.set_ylabel("Progress Percentage")
            plt.tight_layout()
            output_html += fig_to_html(fig)
            plt.close(fig)

    elif tab == 'Individual Progress':
        filtered_df = filter_data_by_date(df, pd.to_datetime(date))
        if filtered_df.empty:
            output_html = "<h3>No individual progress data available.</h3>"
        else:
            figures = create_student_progress_charts(filtered_df)
            output_html += "<h3>Individual Progress</h3><p>These charts show the breakdown of actions taken by each student on the selected date, along with the percentage of their contribution to the total actions.</p>"
            for fig in figures:
                output_html += fig_to_html(fig)
                plt.close(fig)

    elif tab == 'Top Performer':
        filtered_df = filter_data_by_date(df, pd.to_datetime(date))
        if filtered_df.empty:
            output_html = "<h3>No data available for the selected date.</h3>"
        else:
            top_student, top_percentage = find_top_performer(filtered_df)
            output_html = f"<h3>Top Performer</h3><p>The top performer for the selected date is {top_student}, with {top_percentage:.2f}% of the total actions.</p>"

    elif tab == 'Monthly Summary':
        filtered_df = filter_data_by_month(df, pd.to_datetime(date))
        if filtered_df.empty:
            output_html = "<h3>No data available for the selected month.</h3>"
        else:
            fig = create_monthly_summary_chart(filtered_df)
            output_html += "<h3>Monthly Summary</h3><p>This chart shows the overall progress percentage for each student for the selected month.</p>"
            output_html += fig_to_html(fig)
            plt.close(fig)

    # Update the output div with the result
    display(HTML(f"<script>document.getElementById('output').innerHTML = `{output_html}`;</script>"))

# Callback function to search the database
def search_database(student_name, date, tab, document, description, keyword):
    global df
    search_conditions = []
    if student_name:
        search_conditions.append(df['User'].str.contains(student_name, case=False, na=False))
    if date:
        search_conditions.append(df['Time'].dt.strftime('%Y-%m-%d') == date)
    if tab:
        search_conditions.append(df['Tab'].str.contains(tab, case=False, na=False))
    if document:
        search_conditions.append(df['Document'].str.contains(document, case=False, na=False))
    if description:
        search_conditions.append(df['Description'].str.contains(description, case=False, na=False))
    if keyword:
        keyword_condition = df.apply(lambda row: row.astype(str).str.contains(keyword, case=False).any(), axis=1)
        search_conditions.append(keyword_condition)

    if search_conditions:
        result_df = df[search_conditions[0]]
        for condition in search_conditions[1:]:
            result_df = result_df[condition]
    else:
        result_df = df

    if result_df.empty:
        output_html = "<h3>No results found for the given criteria.</h3>"
    else:
        output_html = result_df.to_html(classes='search-results', index=False)
    display(HTML(f"<script>document.getElementById('search-output').innerHTML = `{output_html}`;</script>"))

# Define JavaScript to Python interaction
from google.colab import output
output.register_callback('updateData', update_gui)
output.register_callback('searchDatabase', search_database)
output.register_callback('initFirebase', init_firebase)
output.register_callback('generateReport', generate_report)


# Create widgets for the dashboard
tab_selector = widgets.Dropdown(
    options=['Overall Progress', 'Individual Progress', 'Top Performer', 'Monthly Summary'],
    value='Overall Progress',
    description='Select Tab:',
    style={'description_width': 'initial'}
)

date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False,
    style={'description_width': 'initial'}
)

update_button = widgets.Button(
    description='Show results',
    button_style='success',
    style={'description_width': 'initial'}
)

# HTML layout
html = """
<style>
    body { font-family: Arial, sans-serif; }
    .container { width: 80%; margin: 0 auto; }
    .header { text-align: center; margin-bottom: 20px; position: relative; }
    .form-group { margin-bottom: 20px; }
    .form-group label { display: block; margin-bottom: 5px; }
    .form-group select, .form-group input { width: 100%; padding: 8px; }
    .charts { margin-top: 20px; }
    .chart { margin-bottom: 20px; }
    .upload-container { position: absolute; top: 10px; right: 10px; text-align: right; }
    .upload-container small { display: block; margin-bottom: 5px; }
    .upload-btn { display: inline-block; }
    .search-container { text-align: left; margin-bottom: 20px; }
    .search-btn, .back-btn, .report-gui-btn, .make-report-btn { display: inline-block; margin-bottom: 10px; }
    .back-btn, .search-indexbtn { position: absolute; top: 10px; left: 10px; }
    .report-gui-btn { position: absolute; top: 10px; left: 140px; }
    .search-output { margin-top: 20px; }
    .search-results { width: 100%; border-collapse: collapse; }
    .search-results th, .search-results td { border: 1px solid #ddd; padding: 8px; }
    .search-results th { background-color: #f2f2f2; cursor: pointer; }
    .modal { display: none; position: fixed; z-index: 1; left: 0; top: 0; width: 100%; height: 100%; overflow: auto; background-color: rgb(0,0,0); background-color: rgba(0,0,0,0.4); }
    .modal-content { background-color: #fefefe; margin: 15% auto; padding: 20px; border: 1px solid #888; width: 80%; }
    .centered { text-align: center; }
</style>

<div class="container" id="main-gui">
    <div class="header">
        <h1>Student Progress Dashboard</h1>
        <button id="search-button" class="search-indexbtn">Search Database</button>
        <button id="report-button" class="report-gui-btn">Create Report</button>
        <div class="upload-container">
            <small>Upload credentials.json</small>
            <input type="file" id="firebase-upload" accept=".json" class="upload-btn" title="Upload Credentials" />
        </div>
        <div class="form-group">
            <label for="tab-selector">Select Tab:</label>
            <select id="tab-selector">
                <option value="Overall Progress">Overall Progress</option>
                <option value="Individual Progress">Individual Progress</option>
                <option value="Top Performer">Top Performer</option>
                <option value="Monthly Summary">Monthly Summary</option>
            </select>
        </div>
        <div class="form-group">
            <label for="date-picker">Select Date:</label>
            <input type="date" id="date-picker">
        </div>
        <button id="update-button" class="search-btn">Show results</button>
    </div>
    <div id="output" class="charts"></div>
</div>

<div class="container" id="search-gui" style="display: none;">
    <div class="header">
        <h1>Search Database</h1>
        <div class="form-group">
            <label for="student-name">Student Name:</label>
            <input type="text" id="student-name">
        </div>
        <div class="form-group">
            <label for="search-date-picker">Select Date:</label>
            <input type="date" id="search-date-picker">
        </div>
        <div class="form-group">
            <label for="tab">Tab:</label>
            <input type="text" id="tab">
        </div>
        <div class="form-group">
            <label for="document">Document:</label>
            <input type="text" id="document">
        </div>
        <div class="form-group">
            <label for="description">Description:</label>
            <input type="text" id="description">
        </div>
        <div class="form-group">
            <label for="keyword">Keyword:</label>
            <input type="text" id="keyword">
        </div>
        <div class="centered">
            <button id="search-submit-button" class="search-btn">Search</button>
        </div>
        <div class="upload-container">
            <small>Upload credentials.json</small>
            <input type="file" id="firebase-upload-search" accept=".json" class="upload-btn" title="Upload Credentials" />
        </div>
        <div>
            <button id="back-button" class="back-btn">Back to Main</button>
        </div>
    </div>
    <div id="search-output" class="search-output"></div>
</div>
<div class="container" id="report-gui" style="display: none;">
    <div class="header">
        <h1>Student Performance Report</h1>
        <div class="form-group">
            <label for="student_name">Student Name:</label>
            <input type="text" id="student_name">
        </div>
        <div class="form-group">
            <label for="from_date">From Date:</label>
            <input type="date" id="from_date">
        </div>
        <div class="form-group">
            <label for="to_date">To Date:</label>
            <input type="date" id="to_date">
        </div>
        <div class="centered">
            <button id="make-report-button" class="make-report-btn">Generate Report</button>
        </div>

        <div>
            <button id="back-button2" class="back-btn">Back to Main</button>
        </div>
    </div>
    <div id="search-output" class="search-output"></div>
</div>
<div id="modal" class="modal">
    <div class="modal-content">
        <p id="modal-message"></p>
        <button id="close-modal" class="centered">Close</button>
    </div>
</div>
"""

display(HTML(html))

# JavaScript to handle button clicks and update the GUI
js_code = """
<script>
document.getElementById('update-button').addEventListener('click', () => {
    const tab = document.getElementById('tab-selector').value;
    const date = document.getElementById('date-picker').value;
    google.colab.kernel.invokeFunction('updateData', [tab, date], {});
});

document.getElementById('search-button').addEventListener('click', () => {
    document.getElementById('main-gui').style.display = 'none';
    document.getElementById('search-gui').style.display = 'block';
});

document.getElementById('report-button').addEventListener('click', () => {
    document.getElementById('main-gui').style.display = 'none';
    document.getElementById('report-gui').style.display = 'block';
});

document.getElementById('back-button').addEventListener('click', () => {
    document.getElementById('main-gui').style.display = 'block';
    document.getElementById('search-gui').style.display = 'none';
});

document.getElementById('back-button2').addEventListener('click', () => {
    document.getElementById('main-gui').style.display = 'block';
    document.getElementById('report-gui').style.display = 'none';
});

document.getElementById('search-submit-button').addEventListener('click', () => {
    const studentName = document.getElementById('student-name').value;
    const date = document.getElementById('search-date-picker').value;
    const tab = document.getElementById('tab').value;
    const documentText = document.getElementById('document').value;
    const description = document.getElementById('description').value;
    const keyword = document.getElementById('keyword').value;
    google.colab.kernel.invokeFunction('searchDatabase', [studentName, date, tab, documentText, description, keyword], {});
});

document.getElementById('make-report-button').addEventListener('click', () => {
    const studentName = document.getElementById('student_name').value;
    const fromDate = document.getElementById('from_date').value;
    const toDate = document.getElementById('to_date').value;

    google.colab.kernel.invokeFunction('generateReport', [studentName, fromDate, toDate], {}).then(result => {
        let b64_pdf = result.data['text/plain'];
        b64_pdf = b64_pdf.replace(/^'|'$/g, ''); // Remove leading and trailing single quotes if present
        if (b64_pdf) {
            const link = document.createElement('a');
            link.href = `data:application/pdf;base64,${b64_pdf}`;
            link.download = `${studentName}-${fromDate}-${toDate}.pdf`; // Change the filename here
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        } else {
            alert("Failed to generate the report. Please try again.");
        }
    });
});






function handleFileUpload(event) {
    const file = event.target.files[0];
    const reader = new FileReader();
    reader.onload = function(e) {
        const contents = e.target.result;
        try {
            const jsonContents = JSON.parse(contents);
            google.colab.kernel.invokeFunction('initFirebase', [jsonContents], {});
            showModal('Credentials file uploaded successfully.');
        } catch (error) {
            showModal('Invalid JSON file.');
        }
    };
    reader.readAsText(file);
}

document.getElementById('firebase-upload').addEventListener('change', handleFileUpload);

document.getElementById('firebase-upload-search').addEventListener('change', handleFileUpload);

document.getElementById('close-modal').addEventListener('click', () => {
    document.getElementById('modal').style.display = 'none';
});

function showModal(message) {
    document.getElementById('modal-message').innerText = message;
    document.getElementById('modal').style.display = 'block';
}
</script>
"""

display(HTML(js_code))

# Load initial data from Firebase
try:
    data = fetch_data()
    df = process_data(data)
except Exception as e:
    display(HTML(f'<p style="color: red;">Error loading data: {str(e)}</p>'))


In [7]:
import requests
from bs4 import BeautifulSoup
import re
import firebase_admin
from firebase_admin import credentials, db
from nltk.stem import PorterStemmer
import nltk

nltk.download('punkt')

# Function to fetch and parse the page
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

# Function to index significant words
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

# Remove stop words
def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

# Apply stemming
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

# Function to create an index for the glossary
def create_glossary_index(url):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    return index

# Check if Firebase app is already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate("/content/phoenixes-f3288-firebase-adminsdk-nydgl-2a1de6a8ed.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://phoenixes-34c33-default-rtdb.europe-west1.firebasedatabase.app/'
    })

# URL of the glossary page
url = 'https://en.wikipedia.org/wiki/Glossary_of_architecture'
index = create_glossary_index(url)

# Verify the indexed data
if index:
    print("Indexed data:")
    for word, freq in index.items():
        print(f"Term: {word}, Frequency: {freq}")
else:
    print("Indexing failed. No data found.")

# Convert the index to the required format
index_data = [{'term': word, 'freq': freq} for word, freq in index.items()]

# Save the index data to Firebase
ref = db.reference('glossary_index')
ref.set(index_data)

print("Indexed data has been saved to Firebase.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Indexed data:
Term: glossari, Frequency: 19
Term: of, Frequency: 420
Term: architectur, Frequency: 95
Term: wikipedia, Frequency: 7
Term: jump, Frequency: 1
Term: content, Frequency: 3
Term: main, Frequency: 17
Term: menu, Frequency: 2
Term: move, Frequency: 5
Term: sidebar, Frequency: 3
Term: hide, Frequency: 5
Term: navig, Frequency: 1
Term: pagecontentscurr, Frequency: 1
Term: eventsrandom, Frequency: 1
Term: articleabout, Frequency: 1
Term: wikipediacontact, Frequency: 1
Term: usdon, Frequency: 1
Term: contribut, Frequency: 1
Term: helplearn, Frequency: 1
Term: editcommun, Frequency: 1
Term: portalrec, Frequency: 1
Term: changesupload, Frequency: 2
Term: file, Frequency: 1
Term: search, Frequency: 3
Term: appear, Frequency: 3
Term: creat, Frequency: 7
Term: account, Frequency: 2
Term: log, Frequency: 4
Term: person, Frequency: 1
Term: tool, Frequency: 3
Term: page, Frequency: 6
Term: for, Frequency: 33
Term: out, Frequency: 4
Term: editor, Frequency: 1
Term: learn, Frequency: 1
Ter